## Lexy Feldmann<br>Working with the sqlite3 database

### Import the necessary libraries

In [1]:
# Import all necessary libraries
import pandas as pd
import sqlite3

# Connect to the created database 'finalproject.db'
connect = sqlite3.connect('finalproject.db')

### Grab the Weather report (API data) that is already merged with the HTML data

In [2]:
# Grab the stored dataframe from the previous milestone
%store -r yearly_country_weather

In [3]:
# Add a new column for country and year to have a unique identifer for each country year
yearly_country_weather['Country & Year'] = yearly_country_weather['Country']+' '+yearly_country_weather['Year'].astype(str)

# Print the first 5 rows to view the changes
yearly_country_weather.head()

,Country,Average Temperature (°F),Average Precipitation (in.),Average Sunshine Duration (hrs),Year,Country & Year
0,Andorra,42.628880,0.094111,9.834259,2015,Andorra 2015
1,United Arab Emirates,82.837921,0.001186,10.785155,2015,United Arab Emirates 2015
2,Afghanistan,33.118786,0.048301,10.596569,2015,Afghanistan 2015
3,Antigua and Barbuda,79.113968,0.058527,10.958670,2015,Antigua and Barbuda 2015
4,Anguilla,79.402802,0.046112,10.974804,2015,Anguilla 2015


### Add Yearly_Country_Weather dataframe to sqlite3 database

In [4]:
# Create the new table in the database using the yearly_country_weather dataframe
yearly_country_weather.to_sql('weather_data', connect, if_exists='replace', index=False)

# Verify the weather_data was actually added to the database
query = "SELECT * FROM weather_data"

# Read the dataframe from the database
df_from_db = pd.read_sql(query, connect)

# Print the table to see if it is formatted/created properly
print(df_from_db)

                   Country  Average Temperature (°F)  \
0                  Andorra                 42.628880   
1     United Arab Emirates                 82.837921   
2              Afghanistan                 33.118786   
3      Antigua and Barbuda                 79.113968   
4                 Anguilla                 79.402802   
...                    ...                       ...   
1215                 Yemen                 79.353134   
1216               Mayotte                 77.652557   
1217          South Africa                 67.179626   
1218                Zambia                 70.739273   
1219              Zimbabwe                 70.655540   

      Average Precipitation (in.)  Average Sunshine Duration (hrs)  Year  \
0                        0.094111                         9.834259  2015   
1                        0.001186                        10.785155  2015   
2                        0.048301                        10.596569  2015   
3                      

### Grab the Happiness report (CSV data)

In [5]:
# Grab the stored dataframe from a previous milestone
%store -r happiness_report

In [6]:
# Add a new column for country and year to have a unique identifer for each country year
happiness_report['Country & Year'] = happiness_report['Country']+' '+happiness_report['Year'].astype(str)

# Print the first 5 rows to view the changes
happiness_report.head()

,Country,Happiness Rank,Happiness Score,Year,Country & Year
0,Switzerland,1,7.587,2015,Switzerland 2015
1,Iceland,2,7.561,2015,Iceland 2015
2,Denmark,3,7.527,2015,Denmark 2015
3,Norway,4,7.522,2015,Norway 2015
4,Canada,5,7.427,2015,Canada 2015


### Add Happiness_Report dataframe to sqlite3 database

In [7]:
# Create the new table in the database using the happiness_report dataframe
happiness_report.to_sql('happiness_data', connect, if_exists='replace', index=False)

# Create the new table in the database using the yearly_country_weather dataframe
query = "SELECT * FROM happiness_data"

# Read the dataframe from the database
df_from_db = pd.read_sql(query, connect)

# Print the table to see if it is formatted/created properly
print(df_from_db)

                      Country  Happiness Rank  Happiness Score  Year  \
0                 Switzerland               1            7.587  2015   
1                     Iceland               2            7.561  2015   
2                     Denmark               3            7.527  2015   
3                      Norway               4            7.522  2015   
4                      Canada               5            7.427  2015   
..                        ...             ...              ...   ...   
777                    Rwanda             152            3.334  2019   
778                  Tanzania             153            3.231  2019   
779               Afghanistan             154            3.203  2019   
780  Central African Republic             155            3.083  2019   
781               South Sudan             156            2.853  2019   

                    Country & Year  
0                 Switzerland 2015  
1                     Iceland 2015  
2                     De

### Merge happiness_data and weather_data together in sqlite3

In [8]:
# Create a cursor object from the database we are connected to that will be used to execute SQL commands
cursor = connect.cursor()

# This query specifies each table, happiness_data and weather_data and grabs the specified columns from the first table
# (happiness data) and joins onto the second table (weather data) using the columns in both tables called "Country & Year"
cursor.execute('''
SELECT t1."Year", t1."Country", t1."Happiness Rank", t1."Happiness Score", t2."Average Temperature (°F)", t2."Average Precipitation (in.)", t2."Average Sunshine Duration (hrs)"
FROM happiness_data t1
LEFT JOIN weather_data t2
ON t1."Country & Year" = t2."Country & Year"
''')

# Grabs the merged dataset using the fetchall function and store into a new variable
merged_data = cursor.fetchall()

# Print each row to verify that it worked
for row in merged_data:
    print(row)

# Close the connection to the database
connect.close()

(2015, 'Switzerland', 1, 7.587, 39.69070053100586, 0.20232976973056793, 8.647441864013672)
(2015, 'Iceland', 2, 7.561, 28.527563095092773, 0.10916836559772491, 5.687922954559326)
(2015, 'Denmark', 3, 7.527, 48.07297134399414, 0.10019419342279434, 7.453070163726807)
(2015, 'Norway', 4, 7.522, 35.47833251953125, 0.09597674012184143, 7.245577335357666)
(2015, 'Canada', 5, 7.427, 34.34196853637695, 0.04014671966433525, 8.271017074584961)
(2015, 'Finland', 6, 7.406, 41.500709533691406, 0.07411282509565353, 6.355011940002441)
(2015, 'Netherlands', 7, 7.378, 50.94376754760742, 0.08592387288808823, 7.728980541229248)
(2015, 'Sweden', 8, 7.364, 46.00969696044922, 0.061363402754068375, 7.67929744720459)
(2015, 'New Zealand', 9, 7.286, 56.09467697143555, 0.08473734557628632, 9.537665367126465)
(2015, 'Australia', 10, 7.284, 72.54136657714844, 0.021540295332670212, 10.459875106811523)
(2015, 'Israel', 11, 7.278, 66.81551361083984, 0.01305144838988781, 10.849555015563965)
(2015, 'Costa Rica', 12, 7

In [9]:
# List with the column headers to use in the dataframe
column_headers = ["Year", "Country", "Happiness Rank", "Happiness Score", 
           "Average Temperature (°F)", "Average Precipitation (in.)", "Average Sunshine Duration (hrs)"]

# Create a dataframe using the data from the database, and use the column headers that were just defined
full_data = pd.DataFrame(merged_data, columns=column_headers)

In [10]:
# Get rid of NaN rows in the dataset
full_data = full_data[full_data["Average Temperature (°F)"].notna()]

In [11]:
# Set display rows to show ALL rows when printed
pd.set_option('display.max_rows', None)

# Print merged dataset
full_data

,Year,Country,Happiness Rank,Happiness Score,Average Temperature (°F),Average Precipitation (in.),Average Sunshine Duration (hrs)
0,2015,Switzerland,1,7.587,39.690701,0.202330,8.647442
1,2015,Iceland,2,7.561,28.527563,0.109168,5.687923
2,2015,Denmark,3,7.527,48.072971,0.100194,7.453070
3,2015,Norway,4,7.522,35.478333,0.095977,7.245577
4,2015,Canada,5,7.427,34.341969,0.040147,8.271017
5,2015,Finland,6,7.406,41.500710,0.074113,6.355012
6,2015,Netherlands,7,7.378,50.943768,0.085924,7.728981
7,2015,Sweden,8,7.364,46.009697,0.061363,7.679297
8,2015,New Zealand,9,7.286,56.094677,0.084737,9.537665
9,2015,Australia,10,7.284,72.541367,0.021540,10.459875


In [12]:
# Put full dataset into a csv to be used in the Power BI dashboard
full_data.to_csv('Weather And Happiness.csv')

### Summary & Ethical Implications of My Data Wrangling Steps

<b>Summary of What I Learned</b><br>
From the visualizations created, it was found that temperature, sunshine duration, and precipitation have little to no effect on a country's happiness score. A lot of the happiest countries have colder weather, and from the scatterplot visuals, it seems that as sunshine duration increases, happiness levels decrease. It isn't definitive that happiness levels decrease as sunshine durations and temperatures decrease, but that is what the scatterplots suggest. The first line graph in the PDF suggests, however, that there is a relationship between temperature and happiness, but when you drill into the data and start to cut it in different ways, it tells a different story. More research needs to be done on this topic to truly see the effects of weather on happiness levels.
<br><br><b>Ethical Implications</b><br>
The changes that were made to the data included grabbing each dataset from prior notebooks and adding a column to both that was used in merging the two together. The column added in both was named "Country & Year" and was used as the primary key for both tables. The tables were then loaded into an SQL3 database where the join took place, and then the dataset was spit back out and put into a dataframe to use in creating a CSV file. This CSV file was loaded into Power BI where visualization creation took place. There are no legal or regulatory guidelines for my data or project topic. All data is public and none of it is sensitive. With transforming data in a database through Jupyter Notebook, you run the risk of not correctly manipulating the data (if you don't check your steps as you go). With joins, you also run the risk of not having every value find a partner to merge onto. This was expected and accounted for. I made no assumptions when cleaning the data. My data sources were all public and pulled from trusted sources. It was acquired in an ethical way, as the sites allows for easy access to all data. I would mitigate the ethical implications (if any) by making sure I transformed my data efficiently and effectively - making sure all steps are accounted for and no transformations were messed up by either doing them in the wrong order or incorrectly.